In [1]:
from manipurl.utils.replay_buffer import ReplayBuffer
import numpy as np

replay_buffer = ReplayBuffer(8, 8)

for _ in range(100):
    replay_buffer.insert(np.zeros(8), np.zeros(8), np.zeros(8), 0, 0)

In [2]:
batch = replay_buffer.sample()

In [3]:
print(batch.states.shape)

torch.Size([64, 8])


In [4]:
gamma = 0.99
next_q = np.zeros((64, 1))


print(batch.done.shape)
print(batch.rewards.shape)
print((batch.rewards + gamma * (1 - batch.done) * next_q).shape)

torch.Size([64, 1])
torch.Size([64, 1])
torch.Size([64, 1])


/tmp/ipykernel_140487/1480780003.py:7: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  print((batch.rewards + gamma * (1 - batch.done) * next_q).shape)


In [1]:
import metaworld
import gymnasium as gym


env = gym.make("Meta-World/MT1", env_name="reach-v3")

obs, info = env.reset()

print(env.observation_space.shape[0])
print(env.action_space.shape[0])

obs2, reward, terminated, truncated, info2 = env.step(env.action_space.sample())
env.close()

39
4


/home/dubl/miniconda3/envs/manipurl/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:35: UserWarning: WARN: A Box observation space maximum and minimum values are equal.
  logger.warn("A Box observation space maximum and minimum values are equal.")
/home/dubl/miniconda3/envs/manipurl/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/home/dubl/miniconda3/envs/manipurl/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


In [18]:
print(info2)

{'success': 0.0, 'near_object': 0.3183330416996061, 'grasp_success': 1.0, 'grasp_reward': 0.3183330416996061, 'in_place_reward': 0.12400346342234515, 'obj_to_target': 0.3183330416996061, 'unscaled_reward': 1.2400346342234516}


In [18]:
import torch
import numpy as np


test = []
test.append(torch.tensor(1, device=torch.device("cuda")))
test.append(torch.tensor(3, device=torch.device("cuda")))
test.append(torch.tensor(7))

print(test)
print(torch.as_tensor(test))

[tensor(1, device='cuda:0'), tensor(3, device='cuda:0'), tensor(7)]
tensor([1, 3, 7])


In [14]:
print(torch.as_tensor(4))

tensor(4)


In [3]:
def unfold_seeds(seed_str):
    try:
        sections = [[int(seed) for seed in section.split(',')] for section in seed_str.split('..')]
    except:
        print("Seeds can only be integers.")
        return
    for s_i in range(len(sections) - 1):
        sections[s_i].extend(list(range(sections[s_i][-1] + 1, sections[s_i+1][0])))

    return [seed for section in sections for seed in section]


print(unfold_seeds('1..5,6..8,10'))

[1, 2, 3, 4, 5, 6, 7, 8, 10]


In [6]:
import os

print(os.environ.get('TEST', 'ahem'))

ahem


In [2]:
import torch
import time


model = torch.nn.Sequential(
    torch.nn.Linear(1024, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256)).cuda()

buffer = torch.empty(256).cpu().pin_memory()

start = time.time()

for _ in range(1000):
    x = torch.rand(1024).cuda()
    buffer.copy_(model.forward(x))

mid = time.time()

for _ in range(1000):
    x = torch.rand(1024).cuda()
    output = model.forward(x).cpu()

end = time.time()

print(mid - start, end - mid)

6.773641347885132 7.093922138214111


In [1]:
from manipurl.models.clip import project_text


text_embedding = project_text("testing 123")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [2]:
text_embedding

tensor([[ 8.8553e-02, -3.8899e-02, -1.7174e-01, -4.1013e-03,  5.0780e-02,
         -1.3630e-01, -6.5804e-02, -1.6550e+00, -1.4017e-02,  3.5458e-01,
          4.9149e-02,  1.1292e-01, -1.6047e-01, -6.4814e-02,  1.8676e-01,
          1.6981e-01,  4.1503e-01, -2.3428e-01, -3.0116e-02,  6.0454e-02,
          3.9842e-02, -2.9758e-01,  2.5786e-01, -3.5133e-01, -1.7608e-01,
          1.6531e-01, -2.5653e-01,  3.7034e-01,  1.0868e-02,  4.9361e-02,
         -2.0778e-01, -2.1342e-01,  1.2122e-01, -1.9948e-01,  2.9079e-02,
          2.0002e-01, -3.9868e-02, -1.1064e-01,  3.2888e-02, -2.3359e-01,
          1.0471e-01, -2.2679e-02,  2.2971e-02,  1.4768e-01, -4.8934e-02,
          1.5360e-01, -1.9388e-01,  1.8878e-01, -4.1001e-02, -6.2010e-02,
         -2.6019e-01,  6.1834e-02,  4.4544e-02, -1.2594e-01,  1.0688e-01,
          1.6754e-01,  6.8087e-03,  4.8657e-02, -2.5049e-01, -4.4400e-02,
          2.5696e-01,  1.4922e-01,  2.1289e-01,  7.7035e-02,  1.1201e-01,
          2.0973e-02, -2.8449e-01, -3.

In [3]:
text_emb = project_text('ahh yes...')

In [5]:
text_emb.shape

torch.Size([1, 512])

In [2]:
import manipurl.models.clip as clip

clip.project_text("123")

tensor([[ 1.5655e-01,  6.9829e-02, -1.0981e-01,  2.7771e-01, -7.5693e-02,
         -4.1318e-02, -5.9854e-02, -3.8642e-01, -2.2456e-01,  1.2425e-01,
         -1.5266e-03, -4.2030e-01, -2.3433e-01, -3.9407e-01,  2.9225e-01,
          3.8046e-01,  3.4881e-01, -1.3522e-02, -2.7456e-02,  8.1683e-02,
          1.4102e-01,  1.3750e-01,  2.9109e-01,  1.3883e-01,  8.3824e-02,
          2.1579e-01, -1.9717e-02,  2.2068e-01,  1.1240e-01, -5.8878e-02,
          2.5748e-01, -8.8274e-02,  1.6055e-01, -2.7223e-01,  3.5833e-02,
          3.4241e-01,  9.1353e-02,  5.2306e-02,  3.2261e-01,  3.8243e-02,
         -5.0944e-02, -2.6354e-01,  2.6070e-01,  5.3802e-01,  1.3998e-01,
          1.7604e-01, -3.0883e-02,  2.6781e-01, -3.3841e-01, -5.7023e-02,
         -3.7875e-02,  2.1682e-01, -1.9821e-01,  3.2257e-02, -1.6190e-01,
         -5.6444e-02,  9.3866e-02, -1.1212e-01,  9.7194e-02, -1.2955e-01,
          1.3357e-01, -1.0948e-01,  2.5524e-01, -6.7923e-02,  1.3429e-01,
         -1.4962e-01, -4.4130e-01,  2.

In [1]:
from manipurl.utils.env import create_environment

env = create_environment('button-press-v3', 0)

In [2]:
print(env.observation_space.high)
print(env.observation_space.low)
print(env.observation_space.high == env.observation_space.low)

[5.250e-01 1.025e+00 7.000e-01 1.000e+00       inf       inf       inf
       inf       inf       inf       inf       inf       inf       inf
       inf       inf       inf       inf 5.250e-01 1.025e+00 7.000e-01
 1.000e+00       inf       inf       inf       inf       inf       inf
       inf       inf       inf       inf       inf       inf       inf
       inf 1.000e-05 1.000e-05 1.000e-05]
[-0.525   0.348  -0.0525 -1.        -inf    -inf    -inf    -inf    -inf
    -inf    -inf    -inf    -inf    -inf    -inf    -inf    -inf    -inf
 -0.525   0.348  -0.0525 -1.        -inf    -inf    -inf    -inf    -inf
    -inf    -inf    -inf    -inf    -inf    -inf    -inf    -inf    -inf
  0.      0.      0.    ]
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False]


In [2]:
obs, _ = env.reset()

In [4]:
print(obs)

[ 0.00452851  0.40030765  0.19568645  1.         -0.08331551  0.68460962
  0.115       1.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.00452851  0.40030765  0.19568645  1.         -0.08331551  0.68460962
  0.115       1.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
 -0.08331551  0.77760962  0.115     ]


In [8]:
env.observation_space.low[-3:]

array([0., 0., 0.])